# Phase Lookup (Phaseup) Procedure in Cirq

This advanced notebook explores a symbolic phase lookup mechanism inspired by Gidney (2025). We introduce custom gates for constructing power-product (monomial) registers, applying multi-target classically controlled phases, and uncomputing intermediate data structures.

## 1. Imports

In [ ]:
import cirq
import sympy
from sympy import symbols
from typing import Sequence

## 2. Core Utility Gates
We define reduced versions of the support gates from the sample module (AndGate, MultitargetZGate, MultitargetCZGate, BarrierGate).

In [ ]:
class AndGate(cirq.Gate):
    def _num_qubits_(self): return 3
    def _decompose_(self, qs): yield cirq.CCNOT.on(*qs)
    def _circuit_diagram_info_(self, args): return ['●','●','X']

class MultitargetZGate(cirq.Gate):
    def __init__(self, n_qubits:int, classical_controls):
        self.n_qubits=n_qubits; self.cc = classical_controls
    def _num_qubits_(self): return self.n_qubits
    def _decompose_(self, qs):
        for i in range(self.n_qubits):
            yield cirq.Z.on(qs[i]).with_classical_controls(self.cc[i])
    def _circuit_diagram_info_(self, args):
        return cirq.CircuitDiagramInfo(wire_symbols=tuple(f'Z^({c})' for c in self.cc))

class MultitargetCZGate(cirq.Gate):
    def __init__(self, n_targets:int, classical_controls):
        self.n_targets=n_targets; self.cc=classical_controls
    def _num_qubits_(self): return self.n_targets+1
    def _decompose_(self, qs):
        c, *targets = qs
        for i in range(self.n_targets):
            yield cirq.CZ.on(c, targets[i]).with_classical_controls(self.cc[i])
    def _circuit_diagram_info_(self, args):
        return cirq.CircuitDiagramInfo(wire_symbols=('●',)+tuple(f'Z^({c})' for c in self.cc))

class BarrierGate(cirq.Gate):
    def __init__(self,n:int): self.n=n
    def _num_qubits_(self): return self.n
    def _decompose_(self, qs): return []
    def _circuit_diagram_info_(self, args): return cirq.CircuitDiagramInfo(wire_symbols=('│',)*self.n)

## 3. Power Product Register Construction (DoPowerProductGate)

In [ ]:
class DoPowerProductGate(cirq.Gate):
    def __init__(self, n_vars:int):
        if n_vars < 1: raise ValueError('n_vars >= 1 required')
        self.n_vars=n_vars
        self.n_minterms = 1<<n_vars
        self.n_monomials = self.n_minterms - 1
    def _num_qubits_(self): return self.n_monomials
    def _decompose_(self, qs):
        qs=[None]+list(qs)
        and_gate=AndGate()
        for i in range(1, self.n_vars):
            var_index = 1<<i
            for j in range(1, var_index):
                yield and_gate.on(qs[j], qs[var_index], qs[var_index+j])
    def _circuit_diagram_info_(self,args):
        return cirq.CircuitDiagramInfo(wire_symbols=('PP',)+('.',)*(self.n_monomials-1))
    @classmethod
    def rearrange_qubits(cls, src, aux):
        n=len(src); expected=(1<<n)-n-1
        if len(aux)!=expected: raise ValueError('Wrong aux count')
        out=[None]; next_aux=0
        for qi,q in enumerate(src):
            out.append(q)
            count_new=len(out)-2
            if count_new>0:
                out.extend(aux[next_aux:next_aux+count_new])
                next_aux+=count_new
        return out

## 4. Uncompute Register (UndoPowerProductGate)

In [ ]:
class UndoPowerProductGate(cirq.Gate):
    def __init__(self, tag:str, n_vars:int):
        if n_vars<1: raise ValueError('n_vars>=1')
        self.tag=tag; self.n_vars=n_vars
        self.n_minterms=1<<n_vars; self.n_monomials=self.n_minterms-1
    def _num_qubits_(self): return self.n_monomials
    def _decompose_(self, qs):
        if self.n_minterms<=1: return
        products=[None]+list(qs)
        h=self.n_minterms//2
        while h>1:
            classical_controls=[]
            for i in range(h+1,2*h):
                key=cirq.MeasurementKey(f'{self.tag}_{i}')
                yield cirq.H.on(products[i])
                yield cirq.measure(products[i], key=key.name)
                classical_controls.append(key)
            if h+1<2*h:
                yield BarrierGate(2).on(products[h+1], products[h])
            yield MultitargetCZGate(h-1, classical_controls).on(products[h], *products[1:h])
            h//=2
    def _circuit_diagram_info_(self,args):
        return cirq.CircuitDiagramInfo(wire_symbols=('UPP',)+('.',)*(self.n_monomials-1))

## 5. Boolean Möbius Transform Helper

In [ ]:
from sympy import Xor, Expr
def boole_mobius_transform(exprs):
    n=len(exprs)
    if n & (n-1)!=0: raise ValueError('Length must be power of 2')
    coeffs=list(exprs)
    step=1
    while step<n:
        for i in range(0,n,2*step):
            for j in range(step):
                a=coeffs[i+j]; b=coeffs[i+j+step]
                coeffs[i+j+step]=Xor(a,b,evaluate=False)
        step*=2
    return coeffs

## 6. PhaseupGate: Putting It Together

In [ ]:
class PhaseupGate(cirq.Gate):
    def __init__(self, tag:str, n_vars:int, data):
        if n_vars<1: raise ValueError('n_vars>=1')
        if len(data)!=(1<<n_vars): raise ValueError('data length mismatch')
        self.tag=tag; self.n_vars=n_vars; self.data=data
        self.n_minterms=1<<n_vars; self.n_monomials=self.n_minterms-1
        self.n_aux=self.n_monomials - n_vars
    def get_aux_count(self): return self.n_aux
    def _num_qubits_(self): return self.n_monomials
    def _decompose_(self, qs):
        vars=qs[:self.n_vars]; aux=qs[self.n_vars:]
        mon = DoPowerProductGate.rearrange_qubits(vars, aux)
        coeffs = boole_mobius_transform(self.data)
        yield DoPowerProductGate(self.n_vars).on(*mon[1:])
        yield MultitargetZGate(self.n_monomials, coeffs[1:]).on(*mon[1:])
        yield UndoPowerProductGate(self.tag, self.n_vars).on(*mon[1:])
    def _circuit_diagram_info_(self,args):
        return cirq.CircuitDiagramInfo(wire_symbols=('?ZZ',)*self.n_vars + ('?&ZZ',)*(self.n_monomials-self.n_vars))

## 7. Example Usage

In [ ]:
n=3
data = list(symbols(' '.join(f'x{i}' for i in range(1<<n))))
PU = PhaseupGate('demo', n, data)
orig = cirq.LineQubit.range(n)
aux = cirq.LineQubit.range(n, n+PU.get_aux_count())
op = PU.on(*(orig+aux))
# Decompose for readability (filter away high-level gates)
def keep_fn(op: cirq.Operation):
    g=getattr(op,'gate',None)
    return not isinstance(g, (PhaseupGate, DoPowerProductGate, UndoPowerProductGate))
circuit = cirq.Circuit(cirq.decompose(op, keep=keep_fn))
print(circuit)

## 8. Export to OpenQASM (Optional)
Cirq can export circuits to OpenQASM 3.0 for interoperability.

In [ ]:
print(circuit.to_qasm(version='3.0'))

## 9. Exercises
1. Change `n` to 4 and observe qubit growth.
2. Replace symbolic `data` with explicit Boolean constants (e.g. `sympy.true`, `sympy.false`).
3. Inspect individual decomposed moments to understand measurement + barrier sequencing.